In this I'm going to test BERTopic for topic modeling. It seems to have significant advantages over LDA.

Going to just follow a tutorial and use their dataset for now

In [1]:
from sklearn.datasets import fetch_20newsgroups
from bertopic import BERTopic
import pandas as pd
import pickle

c:\Users\rharter2\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Since BERT uses sentence embeddings in it's process we want the stop words
docs = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
data = pd.Series(docs['data']).sample(5000).to_list() # huely cut down the data because it takes forever otherwise. This is slow but w/e
data[:5]

['[">"= Mark, ">>"= mp]\n\n\n\n    I am sorry you find these charges amusing, Mark. I understand your\nfrustration though--it can be kind of scary to find your assumptions\nchallenged. Some of the specific cultural interference to which I refer\nincludes linguistic manipulation, for instance, their Tzotzil-Spanish\ndictionary removed both Spanish and Tzotzil words for concepts which are\nthreatening to the ruling ideology, e.g., class, conquer, exploitation,\nrepression, revolution, and described words which can express\nideological concepts in examples like "Boss--the boss is good. He treats\nus well and pays us a good wage." As some of my students would say,\n"NOT!"  \n     Your tone implies that you are unlikely to believe me--indeed, why\nshould you? If you are interested enough to do some further research\nthough, and you sound as if you are, here are some references for you.\n \nStoll, David. _Fishers of Men or Founders of Empire? The Wycliffe Bible\nTranslators in Latin America_

In [3]:
# precalculate sentence embeddings so they don't have to be done each time and save them
from sentence_transformers import SentenceTransformer
if 0:
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # fast but pretty accurate embeddings
    embeddings = embedding_model.encode(data, show_progress_bar=True)
    
    with open("embeddings.pkl", "wb") as fOut:
        pickle.dump({'embeddings': embeddings},fOut)



In [10]:
# Load embeddings from file
embeddings = None
if 1:
    with open("embeddings.pkl", "rb") as fIn:
        cache_data = pickle.load(fIn)
        embeddings = cache_data['embeddings']

In [11]:
# Default representations. Want to try another representation that has less stopwords
topicModel = BERTopic(verbose=True, embedding_model="all-MiniLM-L6-v2") 
topics, probs = topicModel.fit_transform(data, embeddings)

2023-10-20 17:17:22,980 - BERTopic - Reduced dimensionality
2023-10-20 17:17:23,159 - BERTopic - Clustered reduced embeddings


In [ ]:
topicModel.save("", serialization='safetensors')

In [15]:
topicModel.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1749,-1_the_to_and_of,"[the, to, and, of, ax, in, that, is, for, it]","[\nSince this is alt.atheism, I hope you don't..."
1,0,492,0_the_to_and_of,"[the, to, and, of, is, it, in, for, have, that]","[\n\n\n\nAs far as I can see, one of the big d..."
2,1,202,1_ax_max_and_the,"[ax, max, and, the, 3t, of, to, that, in, pl]",[Brian Kendig first states:\n\n\nI ask:\n\n\nB...
3,2,190,2_the_is_to_in,"[the, is, to, in, of, that, it, and, you, this]","[\n Mr. Parsli, I have to take exception..."
4,3,189,3_the_to_of_and,"[the, to, of, and, in, is, that, it, you, with]","[Folks,\n\nIt's time to start building some pr..."
...,...,...,...,...,...
61,60,12,60_belief_atheism_xfig_earles,"[belief, atheism, xfig, earles, anovak, umass,...",[As quoted from <1993Apr14.184448.2331@gnv.ifa...
62,61,11,61_uv_gun_to_as,"[uv, gun, to, as, that, imitrex, israeli, the,...",[The people who post to this particular newsgr...
63,62,11,62_we_infallible_reasoning_our,"[we, infallible, reasoning, our, that, rexlex,...",[\nI assumed he was referring to OS/2's 32-bit...
64,63,10,63_fairing_police_frame_odometer,"[fairing, police, frame, odometer, the, horns,...",[---------------------------\nThe federal gove...


In [16]:
topicModel.get_topic(0)

[('the', 0.011695077736286927),
 ('to', 0.010639600726294504),
 ('and', 0.01018664096889342),
 ('of', 0.01000974834133086),
 ('is', 0.009847732805053315),
 ('it', 0.009640403426565889),
 ('in', 0.009396469868430532),
 ('for', 0.009057642135494792),
 ('have', 0.008115122728537703),
 ('that', 0.008049985642428679)]

In [17]:
topicModel.visualize_topics()